In [1]:
import pandas as pd
import sqlite3

In [2]:
# Connecting to sql and cursor object
con = sqlite3.connect('./zippedData/im.db/im.db')
cursor_obj = con.cursor()


In [3]:
%%bash

sqlite3 './zippedData/im.db/im.db'
.schema


CREATE TABLE IF NOT EXISTS "movie_basics" (
"movie_id" TEXT,
  "primary_title" TEXT,
  "original_title" TEXT,
  "start_year" INTEGER,
  "runtime_minutes" REAL,
  "genres" TEXT
);
CREATE TABLE IF NOT EXISTS "directors" (
"movie_id" TEXT,
  "person_id" TEXT
);
CREATE TABLE IF NOT EXISTS "known_for" (
"person_id" TEXT,
  "movie_id" TEXT
);
CREATE TABLE IF NOT EXISTS "movie_akas" (
"movie_id" TEXT,
  "ordering" INTEGER,
  "title" TEXT,
  "region" TEXT,
  "language" TEXT,
  "types" TEXT,
  "attributes" TEXT,
  "is_original_title" REAL
);
CREATE TABLE IF NOT EXISTS "movie_ratings" (
"movie_id" TEXT,
  "averagerating" REAL,
  "numvotes" INTEGER
);
CREATE TABLE IF NOT EXISTS "persons" (
"person_id" TEXT,
  "primary_name" TEXT,
  "birth_year" REAL,
  "death_year" REAL,
  "primary_profession" TEXT
);
CREATE TABLE IF NOT EXISTS "principals" (
"movie_id" TEXT,
  "ordering" INTEGER,
  "person_id" TEXT,
  "category" TEXT,
  "job" TEXT,
  "characters" TEXT
);
CREATE TABLE IF NOT EXISTS "writers" (
"m

In [4]:
# gives all U.S movie titles, year, genres, runtime, important players in the movie, average rating 
# and number of votes on imdb
imdb_df = pd.read_sql ("""
SELECT a.movie_id, TRIM(LOWER(primary_title)) as title,
        start_year as year,
        genres,
        CAST(runtime_minutes as integer) as runtime, d.person_id, 
        primary_name as name,
        category,
        averagerating,
        numvotes
FROM movie_basics AS a 
INNER JOIN movie_akas AS b
ON a.movie_id = b.movie_id
INNER JOIN movie_ratings AS c
ON a.movie_id = c.movie_id
INNER JOIN principals d
ON a.movie_id = d.movie_id
INNER JOIN persons as e
ON d.person_id = e.person_id
WHERE region = 'US' AND numvotes > 1000 AND category IN ('actor', 'actress', 'director', 'producer')
ORDER by averagerating DESC, title 





""", con)

In [5]:
imdb_df.head()

,movie_id,title,year,genres,runtime,person_id,name,category,averagerating,numvotes
0,tt7131622,once upon a time ... in hollywood,2019,"Comedy,Drama",159.0,nm0000093,Brad Pitt,actor,9.7,5600
1,tt7131622,once upon a time ... in hollywood,2019,"Comedy,Drama",159.0,nm0000138,Leonardo DiCaprio,actor,9.7,5600
2,tt7131622,once upon a time ... in hollywood,2019,"Comedy,Drama",159.0,nm0000233,Quentin Tarantino,director,9.7,5600
3,tt7131622,once upon a time ... in hollywood,2019,"Comedy,Drama",159.0,nm0382268,David Heyman,producer,9.7,5600
4,tt7131622,once upon a time ... in hollywood,2019,"Comedy,Drama",159.0,nm0570690,Shannon McIntosh,producer,9.7,5600


In [6]:
# Dropping Duplicates from data frame
imdb_df.drop_duplicates(inplace = True)

In [7]:
# Checking for Null Vallues
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34929 entries, 0 to 46029
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       34929 non-null  object 
 1   title          34929 non-null  object 
 2   year           34929 non-null  int64  
 3   genres         34926 non-null  object 
 4   runtime        34923 non-null  float64
 5   person_id      34929 non-null  object 
 6   name           34929 non-null  object 
 7   category       34929 non-null  object 
 8   averagerating  34929 non-null  float64
 9   numvotes       34929 non-null  int64  
dtypes: float64(2), int64(2), object(6)
memory usage: 2.9+ MB


In [8]:
# Dropping Null Values
imdb_df.dropna(inplace = True)

In [9]:
# Checking the number of each category
imdb_df['category'].value_counts()

actor       11826
producer    10274
actress      7338
director     5482
Name: category, dtype: int64

In [11]:
# Checking out descriptive statistics to mak sure nothing is out of the ordinary
imdb_df.describe()

,year,runtime,averagerating,numvotes
count,34920.000000,34920.000000,34920.000000,3.492000e+04
mean,2014.297509,102.435653,6.008826,4.385715e+04
std,2.603356,18.720611,1.146232,1.057586e+05
min,2010.000000,40.000000,1.300000,1.001000e+03
25%,2012.000000,90.000000,5.300000,2.180500e+03
50%,2014.000000,98.000000,6.100000,6.457000e+03
75%,2017.000000,110.000000,6.800000,3.213500e+04
max,2019.000000,467.000000,9.700000,1.841066e+06


In [12]:
imdb_df['genres'] = imdb_df['genres'].str.split(',')
imdb_df = imdb_df.explode('genres')

In [13]:
imdb_df.reset_index(drop = 'index', inplace = True)

In [28]:
imdb_df.head(100)

,movie_id,title,year,genres,runtime,person_id,name,category,averagerating,numvotes
0,tt7131622,once upon a time ... in hollywood,2019,Comedy,159.0,nm0000093,Brad Pitt,actor,9.7,5600
1,tt7131622,once upon a time ... in hollywood,2019,Drama,159.0,nm0000093,Brad Pitt,actor,9.7,5600
2,tt7131622,once upon a time ... in hollywood,2019,Comedy,159.0,nm0000138,Leonardo DiCaprio,actor,9.7,5600
3,tt7131622,once upon a time ... in hollywood,2019,Drama,159.0,nm0000138,Leonardo DiCaprio,actor,9.7,5600
4,tt7131622,once upon a time ... in hollywood,2019,Comedy,159.0,nm0000233,Quentin Tarantino,director,9.7,5600
...,...,...,...,...,...,...,...,...,...,...
95,tt4154796,avengers: endgame,2019,Action,181.0,nm0000375,Robert Downey Jr.,actor,8.8,441135
96,tt4154796,avengers: endgame,2019,Adventure,181.0,nm0000375,Robert Downey Jr.,actor,8.8,441135
97,tt4154796,avengers: endgame,2019,Sci-Fi,181.0,nm0000375,Robert Downey Jr.,actor,8.8,441135
98,tt4154796,avengers: endgame,2019,Action,181.0,nm0262635,Chris Evans,actor,8.8,441135


In [15]:
imdb_df.to_csv('imdb_df.csv', index = False)